In [2]:
import pandas as pd
import os
import numpy as np


In [3]:
df= pd.DataFrame(columns=['Title', 'Source', 'Format', 'Type', 'isPartOf', 'isFormatOf', 'isRequiredBy'])
resource_container='Course_Outline_[EECS4462]'
path= './WebCrawlerOutput/'+resource_container


## Adding the ER from the transcripts and audio files

In [9]:
path='./Lecture_Audio/'
formats={'.txt':'mER','.wav':'mER','.mp4':'iER'}
is_part_of={'.txt':'Lecture_Transcripts','.wav':'Lecture_Audio','.mp4':'Lecture_Video'}
is_format_of={'.txt':'.wav','.wav':'.mp4','.mp4':None}
for file in os.listdir(path):

    for format,type in formats.items():
        new_row={
            'Title': file,
            'Source':'EECS4462',
            'Format': format,
            'Type':type,
            'isPartOf':is_part_of[format],
            'isFormatOf':('.'.join(file.split('.')[:-1]) + is_format_of[format] )if not format=='.mp4' else ''
            # 'isRequiredBy':0
        }

        df = pd.concat([pd.DataFrame(new_row,index=[0]),df])
        df=df.reset_index(drop=True)

In [10]:
df


,Title,Source,Format,Type,isPartOf,isFormatOf,isRequiredBy
0,A2StarterCode-s1-full.wav,EECS4462,.mp4,iER,Lecture_Video,,NaN
1,A2StarterCode-s1-full.wav,EECS4462,.wav,mER,Lecture_Audio,A2StarterCode-s1-full.mp4,NaN
2,A2StarterCode-s1-full.wav,EECS4462,.txt,mER,Lecture_Transcripts,A2StarterCode-s1-full.wav,NaN
3,Untitled-s1-full_(8).wav,EECS4462,.mp4,iER,Lecture_Video,,NaN
4,Untitled-s1-full_(8).wav,EECS4462,.wav,mER,Lecture_Audio,Untitled-s1-full_(8).mp4,NaN
...,...,...,...,...,...,...,...
71,Untitled-s1-full_(10).wav,EECS4462,.txt,mER,Lecture_Transcripts,Untitled-s1-full_(10).wav,NaN
72,Untitled-s1-full_(10).wav,EECS4462,.wav,mER,Lecture_Audio,Untitled-s1-full_(10).mp4,NaN
73,Untitled-s1-full_(10).wav,EECS4462,.txt,mER,Lecture_Transcripts,Untitled-s1-full_(10).wav,NaN
74,Untitled-s1-full_(10).wav,EECS4462,.wav,mER,Lecture_Audio,Untitled-s1-full_(10).mp4,NaN


## Generate all weeks in the outline for both ER ==> iER and rER

Separate each week

Manually split the different ER

In [3]:
outline = open(path+'/'+'Course_Outline_[EECS4462]_content.txt',mode='r', encoding='utf-8').read().split('\n')
# outline.find('−Table of Contents')

# outline.index('−Table of Contents')
begining_weeks = outline.index('−Table of Contents')+2
ending_weeks = outline[begining_weeks:].index('Course Outline')+2

weeks = outline[begining_weeks+ending_weeks:]

for week_no in range(12,0,-1):
    week_begins_index = [index for index,item in enumerate(weeks) if 'Week '+str(week_no)+' ' in item][0] if week_no>1 else week_ends_index
    week_ends_index = [index for index,item in enumerate(weeks) if 'Week '+str(week_no-1)+' ' in item][0] if week_no>1 else -1

    week_studyM_index = [index for index,item in enumerate(weeks[week_begins_index:week_ends_index]) if 'Study material' in item][0] 
    week_weeklyG_index = [index for index,item in enumerate(weeks[week_begins_index:week_ends_index]) if 'Weekly goals' in item][0] 

    # week_due = open(path+'/'+'_week_'+str(week_no), mode='w', encoding='utf-8')
    # week_due.write("\n".join(weeks[week_begins_index:week_ends_index]))
    # week_due.close()

    week_due = open(path+'/'+resource_container+'_week_'+str(week_no)+'_due.txt', mode='w', encoding='utf-8')
    week_due.write(weeks[ week_begins_index])
    week_due.write("\n".join(weeks[week_begins_index : week_begins_index + week_studyM_index]))
    week_due.close()

    week_studyM = open(path+'/'+resource_container+'_week_'+str(week_no)+'_study_material.txt', mode='w', encoding='utf-8')
    week_studyM.write(weeks[ week_begins_index])
    week_studyM.write("\n".join(weeks[ week_begins_index + week_studyM_index : week_begins_index + week_weeklyG_index]))
    week_studyM.close()

    week_weeklyG = open(path+'/'+resource_container+'_week_'+str(week_no)+'_weekly_goals.txt', mode='w', encoding='utf-8')
    week_weeklyG.write(weeks[ week_begins_index])
    week_weeklyG.write("\n".join(weeks[week_begins_index + week_weeklyG_index: week_ends_index]))
    week_weeklyG.close()



## Populate DF

In [3]:
def populate_df(resource_container,path,df):
    dict_type={
        'due.txt':'rER',
        'goals.txt':'rER',
        'material.txt': 'iER'
    }
    for item in os.listdir(path):

        # If it is not the file content.txt or link.txt assume will be part of the content
        # Otherwise those two files are part of the container (the folder itself or webpage)
        is_part_of = resource_container if item.__contains__('content.txt') or item.__contains__('links.txt') else resource_container+'_content.txt'
        type = dict_type[item.split('_')[-1]] if item.split('_')[-1] in dict_type.keys() else None
        new_row={
            'Title': item,
            'Source':'EECS4462',
            'Format': item[-4:],
            'Type':type,
            'isPartOf':is_part_of
            # 'isFormatOf':0,
            # 'isRequiredBy':0
        }
        df = pd.concat([pd.DataFrame(new_row,index=[0]),df])
        df=df.reset_index(drop=True)
    return df
        



In [4]:
os.listdir('./WebCrawlerOutput/')

['Resources_[EECS4462]',
 'Course_Outline_[EECS4462]',
 'Assignment_2_[EECS4462]',
 'eval_[EECS4462]',
 'Assignment_1_[EECS4462]',
 'Instructor_[EECS4462]',
 'Important_Dates_[EECS4462]',
 'Assignment_3_[EECS4462]',
 'Grades_[EECS4462]']

In [5]:
for resource_container in os.listdir('./WebCrawlerOutput/'):
    path= './WebCrawlerOutput/'+resource_container
    new_row={
            'Title': resource_container,
            'Source':'EECS4462',
            # 'Format': item[-4:],
            'Type':'container',
            # 'isPartOf':is_part_of
            # 'isFormatOf':0,
            # 'isRequiredBy':0
        }
    df = pd.concat([pd.DataFrame(new_row,index=[0]),df])
    df = df.reset_index(drop=True)
    df = populate_df(resource_container,path,df)

In [6]:
df

,Title,Source,Format,Type,isPartOf,isFormatOf,isRequiredBy
0,Grades_[EECS4462]_links.txt,EECS4462,.txt,None,Grades_[EECS4462],NaN,NaN
1,Grades_[EECS4462]_content.txt,EECS4462,.txt,None,Grades_[EECS4462],NaN,NaN
2,Grades_[EECS4462],EECS4462,NaN,container,NaN,NaN,NaN
3,Assignment_3_[EECS4462]_content.txt,EECS4462,.txt,None,Assignment_3_[EECS4462],NaN,NaN
4,Assignment_3_[EECS4462]_links.txt,EECS4462,.txt,None,Assignment_3_[EECS4462],NaN,NaN
...,...,...,...,...,...,...,...
76,Course_Outline_[EECS4462],EECS4462,NaN,container,NaN,NaN,NaN
77,assignments:assignment4.pdf,EECS4462,.pdf,None,Resources_[EECS4462]_content.txt,NaN,NaN
78,Resources_[EECS4462]_content.txt,EECS4462,.txt,None,Resources_[EECS4462],NaN,NaN
79,Resources_[EECS4462]_links.txt,EECS4462,.txt,None,Resources_[EECS4462],NaN,NaN


In [11]:
df.to_csv('./eecs4462_generated_trasncripts.csv')
